<a href="https://colab.research.google.com/github/bvanitsthian/cfpb_text_classification-el_chapo/blob/bua_branch/complaints_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

In [ ]:
nltk.download('punkt')

In [ ]:
nltk.download('stopwords')

In [2]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
complaints = pd.read_csv('/content/drive/MyDrive/Classification of Consumer Complaints/complaints.csv', index_col = None)

In [9]:
complaints

,Consumer complaint narrative,Issue
0,My name is XXXX XXXX this complaint is not mad...,Incorrect information on your report
1,I searched on XXXX for XXXXXXXX XXXX and was ...,Fraud or scam
2,I have a particular account that is stating th...,Incorrect information on your report
3,I have not supplied proof under the doctrine o...,Attempts to collect debt not owed
4,Hello i'm writing regarding account on my cred...,Incorrect information on your report
...,...,...
353427,Collections account I have no knowledge of,Attempts to collect debt not owed
353428,"Dear CFPB Team, The reason for my complaint is...",Attempts to collect debt not owed
353429,FRCA violations : Failing to Follow Debt Dispu...,Attempts to collect debt not owed
353430,"My Father, a XXXX XXXX acquired an HECM rever...",Struggling to pay mortgage


#**Text Preprocessing**

In [10]:
complaints['Consumer complaint narrative'] = complaints['Consumer complaint narrative'].str.lower()

In [13]:
complaints['Consumer complaint narrative']

0         my name is xxxx xxxx this complaint is not mad...
1         i searched on xxxx for xxxxxxxx xxxx  and was ...
2         i have a particular account that is stating th...
3         i have not supplied proof under the doctrine o...
4         hello i'm writing regarding account on my cred...
                                ...                        
353427           collections account i have no knowledge of
353428    dear cfpb team, the reason for my complaint is...
353429    frca violations : failing to follow debt dispu...
353430    my father, a xxxx xxxx  acquired an hecm rever...
353431    i have tried to contact cash app about a fraud...
Name: Consumer complaint narrative, Length: 353432, dtype: object

###**Text Tokenization [https://towardsdatascience.com/5-simple-ways-to-tokenize-text-in-python-92c6804edfc4#31ad](https://)**

In [ ]:
# takes too long to run # complaints["Consumer complaint narrative"] = complaints["Consumer complaint narrative"].apply(word_tokenize)

In [19]:
tokenizer = RegexpTokenizer(r'\w+') #\w+ matches one or more word characters
complaints['Consumer complaint narrative'] = complaints['Consumer complaint narrative'].apply(tokenizer.tokenize)

In [22]:
stop_words = set(stopwords.words('english')) #create english stopwords
complaints['Consumer complaint narrative'] = complaints['Consumer complaint narrative'] \
          .apply(lambda text: [word for word in text if word.lower() not in stop_words]) #remove the stop words from each list of tokens in the column

In [ ]:
stemmer = PorterStemmer() #reduce words to their root 
complaints['Consumer complaint narrative'] = complaints['Consumer complaint narrative'].apply(lambda text: [stemmer.stem(word) for word in text])

In [ ]:
complaints['Consumer complaint narrative'] = complaints['Consumer complaint narrative'].apply(lambda text: ' '.join(text))